# 人工知能 csc.t272
## プロジェクト課題
### MNIST dataset をもちいた文字生成

In [ ]:
# パッケージのインポート
import numpy as np
import torch
import torch.nn as nn
import torchvision
import matplotlib.pyplot as plt

In [ ]:
# MNIST dataset の準備
train_data = torchvision.datasets.MNIST(train=True)
test_data = torchvision.datasets.MNIST(train=False)

train = torch.utils.data.DataLoader(train_data, batch_size=bs)
test = torch.utils.data.DataLoader(test_data, batch_size=bs)

In [ ]:
# model の作成
# MNIST dataset; [28, 28]
class MNIST_NN(nn.Module):
    def __init__(
            self,
            layer_dim=1024,
    ):
        super().__init__()
        self.layer1 = nn.Linear(784, layer_dim)
        self.layer2 = nn.Linear(layer_dim, layer_dim)
        self.layer3 = nn.Linear(layer_dim, 10)
        self.act = nn.ReLU()

    def forward(self, input):
        x = input.reshape([784, 1])
        x = self.layer1(x)
        x = self.act(x)
        x = self.layer2(x)
        x = self.act(x)
        x = self.layer3(x)
        pred = nn.Softmax(x)
        return pred





In [ ]:
# 学習
model = MNIST_NN()
bs = 4
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters())